In [1]:
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-1L5H5E7\\MSSQLSERVER02;"  # Instance الصحيح
    "DATABASE=smallhospital;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()
cursor.execute("SELECT 1")
print(cursor.fetchone())  # يجب أن يظهر (1,)
cursor.close()
conn.close()


(1,)


In [4]:
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-1L5H5E7\\MSSQLSERVER02;"
    "DATABASE=smallhospital;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

ddl_script = """
-- Branch
CREATE TABLE Branch (
    BranchID INT PRIMARY KEY,
    Name NVARCHAR(100) NOT NULL,
    City NVARCHAR(100),
    Address NVARCHAR(200),
    Phone NVARCHAR(50),
    OpeningDate DATE
);
-- Department
CREATE TABLE Department (
    DepartmentID INT PRIMARY KEY,
    Name NVARCHAR(100) NOT NULL,
    BranchID INT NOT NULL,
    CONSTRAINT FK_Department_Branch FOREIGN KEY (BranchID) REFERENCES Branch(BranchID)
);
-- Staff
CREATE TABLE Staff (
    StaffID INT PRIMARY KEY,
    Name NVARCHAR(100) NOT NULL,
    Email NVARCHAR(100) UNIQUE,
    Gender CHAR(1) CHECK (Gender IN ('M','F')),
    Phone NVARCHAR(50),
    Position NVARCHAR(100),
    Salary DECIMAL(10,2),
    DepartmentID INT NOT NULL,
    CONSTRAINT FK_Staff_Department FOREIGN KEY (DepartmentID) REFERENCES Department(DepartmentID)
);
-- Doctor
CREATE TABLE Doctor (
    DoctorID INT PRIMARY KEY,
    StaffID INT NOT NULL,
    Specialty NVARCHAR(100),
    LicenseNumber NVARCHAR(50) UNIQUE,
    ConsultationFee DECIMAL(10,2),
    CONSTRAINT FK_Doctor_Staff FOREIGN KEY (StaffID) REFERENCES Staff(StaffID)
);
-- Patient
CREATE TABLE Patient (
    PatientID INT PRIMARY KEY,
    Name NVARCHAR(100) NOT NULL,
    Email NVARCHAR(100),
    Gender CHAR(1) CHECK (Gender IN ('M','F')),
    BirthDate DATE,
    Phone NVARCHAR(50),
    Address NVARCHAR(200)
);
-- Appointment
CREATE TABLE Appointment (
    AppointmentID INT PRIMARY KEY,
    PatientID INT NOT NULL,
    DoctorID INT NOT NULL,
    AppointmentDate DATETIME,
    Status NVARCHAR(50) CHECK (Status IN ('Scheduled','Completed','Cancelled','NoShow')),
    CONSTRAINT FK_Appointment_Patient FOREIGN KEY (PatientID) REFERENCES Patient(PatientID),
    CONSTRAINT FK_Appointment_Doctor FOREIGN KEY (DoctorID) REFERENCES Doctor(DoctorID)
);
-- Admission
CREATE TABLE Admission (
    AdmissionID INT PRIMARY KEY,
    PatientID INT NOT NULL,
    BranchID INT NOT NULL,
    AdmissionDate DATETIME,
    DischargeDate DATETIME,
    CONSTRAINT FK_Admission_Patient FOREIGN KEY (PatientID) REFERENCES Patient(PatientID),
    CONSTRAINT FK_Admission_Branch FOREIGN KEY (BranchID) REFERENCES Branch(BranchID)
);
-- Room
CREATE TABLE Room (
    RoomID INT PRIMARY KEY,
    BranchID INT NOT NULL,
    RoomType NVARCHAR(50),
    CONSTRAINT FK_Room_Branch FOREIGN KEY (BranchID) REFERENCES Branch(BranchID)
);
-- Bed
CREATE TABLE Bed (
    BedID INT PRIMARY KEY,
    RoomID INT NOT NULL,
    IsAvailable BIT DEFAULT 1,
    CONSTRAINT FK_Bed_Room FOREIGN KEY (RoomID) REFERENCES Room(RoomID)
);
-- Insurance
CREATE TABLE Insurance (
    InsuranceID INT PRIMARY KEY,
    PatientID INT NOT NULL,
    Provider NVARCHAR(100),
    PolicyNumber NVARCHAR(50),
    CoverageAmount DECIMAL(10,2),
    CONSTRAINT FK_Insurance_Patient FOREIGN KEY (PatientID) REFERENCES Patient(PatientID)
);
-- Invoice
CREATE TABLE Invoice (
    InvoiceID INT PRIMARY KEY,
    PatientID INT NOT NULL,
    InvoiceDate DATE,
    TotalAmount DECIMAL(10,2),
    CONSTRAINT FK_Invoice_Patient FOREIGN KEY (PatientID) REFERENCES Patient(PatientID)
);
-- Payment
CREATE TABLE Payment (
    PaymentID INT PRIMARY KEY,
    InvoiceID INT NOT NULL,
    PaymentDate DATE,
    Amount DECIMAL(10,2),
    CONSTRAINT FK_Payment_Invoice FOREIGN KEY (InvoiceID) REFERENCES Invoice(InvoiceID)
);
-- BedAssignment
CREATE TABLE BedAssignment (
    BedAssignmentID INT PRIMARY KEY,
    AdmissionID INT NOT NULL,
    BedID INT NOT NULL,
    AssignDate DATETIME,
    ReleaseDate DATETIME,
    CONSTRAINT FK_BedAssignment_Admission FOREIGN KEY (AdmissionID) REFERENCES Admission(AdmissionID),
    CONSTRAINT FK_BedAssignment_Bed FOREIGN KEY (BedID) REFERENCES Bed(BedID)
);
-- Service
CREATE TABLE Service (
    ServiceID INT PRIMARY KEY,
    Name NVARCHAR(100),
    Fee DECIMAL(10,2)
);
-- InvoiceLine
CREATE TABLE InvoiceLine (
    InvoiceLineID INT PRIMARY KEY,
    InvoiceID INT NOT NULL,
    ServiceID INT NOT NULL,
    Quantity INT,
    Price DECIMAL(10,2),
    CONSTRAINT FK_InvoiceLine_Invoice FOREIGN KEY (InvoiceID) REFERENCES Invoice(InvoiceID),
    CONSTRAINT FK_InvoiceLine_Service FOREIGN KEY (ServiceID) REFERENCES Service(ServiceID)
);
"""

# تنفيذ السكريبت
for statement in ddl_script.split(";"):
    if statement.strip():
        cursor.execute(statement)

conn.commit()
cursor.close()
conn.close()
print("✅ All 15 tables created successfully in SQL Server!")


✅ All 15 tables created successfully in SQL Server!


In [6]:
import pyodbc
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()
arabic_names = [
    "Ahmed Mohamed","Mahmoud Ali","Omar Hassan","Youssef Ibrahim",
    "Mohamed Adel","Khaled Mostafa","Sara Ahmed","Aya Mohamed",
    "Mona Hassan","Nour Ali","Salma Ibrahim","Esraa Adel"
]

# ===== اتصال بقاعدة البيانات =====
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-1L5H5E7\\MSSQLSERVER02;"
    "DATABASE=smallhospital;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

# ===== Branch =====
branches = [(i,f"Hospital Branch {i}",random.choice(["Cairo","Giza","Alexandria","Mansoura","Tanta"]),fake.address(),fake.phone_number(),fake.date_this_decade()) for i in range(1,6)]
cursor.executemany("INSERT INTO Branch VALUES (?,?,?,?,?,?)", branches)

# ===== Department =====
dept_names = ["Cardiology","Neurology","Orthopedic","ICU","ER","Pediatrics","Radiology","Oncology","Surgery","Internal"]
departments = [(i,dept_names[i-1],random.randint(1,5)) for i in range(1,11)]
cursor.executemany("INSERT INTO Department VALUES (?,?,?)", departments)

# ===== Staff =====
staff = []
for i in range(1,121):
    name = random.choice(arabic_names)
    unique_email = f"{name.replace(' ','').lower()}{i}@hospital.com"  # فريد
    staff.append((
        i,
        name,
        unique_email,
        random.choice(["M","F"]),
        fake.phone_number(),
        random.choice(["Admin","Doctor","Reception"]),
        random.randint(4000,15000),
        random.randint(1,10)
    ))
cursor.executemany("INSERT INTO Staff VALUES (?,?,?,?,?,?,?,?)", staff)

# ===== Doctor =====
doctors = [(i,i,random.choice(["Cardiology","Neurology","Surgery","Pediatrics","Orthopedic"]),f"LIC-{1000+i}",random.randint(200,600)) for i in range(1,41)]
cursor.executemany("INSERT INTO Doctor VALUES (?,?,?,?,?)", doctors)

# ===== Patient =====
patients = []
for i in range(1,301):
    name = random.choice(arabic_names)
    unique_email = f"{name.replace(' ','').lower()}{i}@gmail.com"
    patients.append((i,name,unique_email,random.choice(["M","F"]),fake.date_of_birth(minimum_age=1, maximum_age=80),fake.phone_number(),fake.address()))
cursor.executemany("INSERT INTO Patient VALUES (?,?,?,?,?,?,?)", patients)

# ===== Appointment =====
appointments = [(i,random.randint(1,300),random.randint(1,40),fake.date_time_this_year(),random.choice(["Scheduled","Completed","Cancelled","NoShow"])) for i in range(1,801)]
cursor.executemany("INSERT INTO Appointment VALUES (?,?,?,?,?)", appointments)

# ===== Admission =====
admissions = []
for i in range(1,501):
    ad = fake.date_time_this_year()
    admissions.append((i,random.randint(1,300),random.randint(1,5),ad,ad + timedelta(days=random.randint(1,10))))
cursor.executemany("INSERT INTO Admission VALUES (?,?,?,?,?)", admissions)

# ===== Room & Bed =====
rooms, beds = [], []
room_id = 1
bed_id = 1
for b in range(1,6):
    for r in range(12):
        rooms.append((room_id,b,random.choice(["ICU","Normal","VIP"])))
        for _ in range(2):
            beds.append((bed_id,room_id,1))
            bed_id += 1
        room_id += 1
cursor.executemany("INSERT INTO Room VALUES (?,?,?)", rooms)
cursor.executemany("INSERT INTO Bed VALUES (?,?,?)", beds)

# ===== BedAssignment =====
assignments = [(i,i,random.randint(1,120),fake.date_time_this_year(),fake.date_time_this_year() + timedelta(days=random.randint(1,7))) for i in range(1,501)]
cursor.executemany("INSERT INTO BedAssignment VALUES (?,?,?,?,?)", assignments)

# ===== Service =====
service_names = ["X-Ray","MRI","Blood Test","CT Scan","ECG","Consultation"]
services = [(i,random.choice(service_names),random.randint(100,1000)) for i in range(1,16)]
cursor.executemany("INSERT INTO Service VALUES (?,?,?)", services)

# ===== Invoice / InvoiceLine / Payment =====
invoices, lines, payments = [], [], []
line_id = 1
for i in range(1,501):
    invoices.append((i,random.randint(1,300),fake.date_this_year(),0))
    payments.append((i,i,fake.date_this_year(),random.randint(300,5000)))
    for _ in range(random.randint(1,3)):
        lines.append((line_id,i,random.randint(1,15),random.randint(1,3),random.randint(100,1000)))
        line_id += 1
cursor.executemany("INSERT INTO Invoice VALUES (?,?,?,?)", invoices)
cursor.executemany("INSERT INTO InvoiceLine VALUES (?,?,?,?,?)", lines)
cursor.executemany("INSERT INTO Payment VALUES (?,?,?,?)", payments)

# ===== Insurance =====
ins = [(i,random.randint(1,300),"Misr Insurance",f"POL-{1000+i}",random.randint(10000,100000)) for i in range(1,201)]
cursor.executemany("INSERT INTO Insurance VALUES (?,?,?,?,?)", ins)

# ===== Commit & Close =====
conn.commit()
cursor.close()
conn.close()
print("✅ All 15 tables populated successfully with realistic data and unique emails!")


✅ All 15 tables populated successfully with realistic data and unique emails!
